In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score, make_scorer
)

1. Побудувати моделi регресiї згiдно з варiантом.
2. Виконати прогнози на основi побудованих моделей.
3. Для кожної з моделей оцiнити, чи має мiсце перенавчання.
4. Розрахувати додатковi результати моделей, наприклад, апостерiорнi iмовiрностi, опорнi вектори або iншi (згiдно з варiантом).
7. В задачах регресiї розрахувати критерiї якостi для кожної моделi окремо
на навчальнiй та перевiрочнiй множинах:
- коефiцiєнт детермiнацiї R2
- помилки RMSE, MAE та MAPE.
8. Виконати решiтчастий пошук (grid search) для пiдбору гiперпараметрiв
моделей.
9. Зробити висновки про якiсть роботи моделей на дослiджених даних. На
основi критерiїв якостi вибрати найкращу модель.
10. Навчити моделi на пiдмножинах навчальних даних. Оцiнити, наскiльки
розмiр навчальної множини впливає на якiсть моделi.

# Get metrics without cross validation

```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

lin_reg_metrics = {
    "rmse": mean_squared_error(y_test, y_pred, squared=False),
    "mae": mean_absolute_error(y_test, y_pred),
    "mape": mean_absolute_percentage_error(y_test, y_pred),
    "r2": r2_score(y_test, y_pred)
}
lin_reg_metrics
```

# Loading data and create scores

In [22]:
# load dataset
X, y = load_diabetes(return_X_y=True)

# scoring for models
scoring={
    "rmse": make_scorer(root_mean_squared_error, greater_is_better=False),
    "mae": make_scorer(mean_absolute_error, greater_is_better=False),
    "mape": make_scorer(mean_absolute_percentage_error, greater_is_better=False),
    "r2": make_scorer(r2_score, greater_is_better=False),
}

In [23]:
def root_mean_squared_error(*args, **kwargs):
    return mean_squared_error(squared=False, *args, **kwargs)

def get_metrics(model, X=X, y=y, scoring=scoring, cv=10, get_mean=True):
    metrics = cross_validate(model, X, y, scoring=scoring, cv=cv)
    metrics = {k: np.abs(v) for k, v in metrics.items()}
    metrics_mean = {k: np.mean(v) for k, v in metrics.items()}
    
    ret_val = metrics_mean if get_mean else metrics
    return ret_val

# Linear Regression model

In [24]:
get_metrics(LinearRegression())

{'fit_time': 0.000648331642150879,
 'score_time': 0.0007868289947509765,
 'test_rmse': 54.40461553640237,
 'test_mae': 44.22301077001687,
 'test_mape': 0.39455077012294637,
 'test_r2': 0.461962361958337}

# Ridge Regression model

In [25]:
get_metrics(Ridge(alpha=0.5))

{'fit_time': 0.0006236553192138672,
 'score_time': 0.000793004035949707,
 'test_rmse': 55.79111301890865,
 'test_mae': 46.24495525021393,
 'test_mape': 0.42058494897932996,
 'test_r2': 0.44180819270734994}

# Linear Regression model with polynomial features